<a href="https://colab.research.google.com/github/radonys/webscrapers/blob/master/Pinterest_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pinterest Scraper

## Install Modules

In [0]:
!pip install bs4 selenium pandas scikit-learn

## Import Modules

In [0]:
# For accessing and parsing Pinterest Webpages
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from urllib.request import urlopen
import json

# Structuring Collected Data
import pandas as pd

# General Libraries
import time
import re
import os
import requests
import shutil
import pickle

## Install Chrome Driver for scrolling Instagram pages

References:

1) How can we use Selenium Webdriver in colab.research.google.com? [[StackOverFlow](https://stackoverflow.com/questions/51046454/how-can-we-use-selenium-webdriver-in-colab-research-google-com)]

In [0]:
!apt install chromium-chromedriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

## Variable Declarations


In [0]:
browser = webdriver.Chrome('chromedriver',options=options)

#Pinterest Users to be scraped
#'amsigfurniture', 'ashleyhomestore', 'bonobos', 'cocacola', 'ethanallen', 'IKEAUSA', 'lazboy', 'potterybarn', 'raymourflanigan', 'simplygum', 'StarFurniture',
pinterest_users = ['williamssonoma', 'walmart']

## Pinterest Login

**Insert the Login ID and Password for Login.**

References:

1) Pinterest Image Scraper ([GitHub](https://raw.githubusercontent.com/xjdeng/pinterest-image-scraper/538e18f520cad8051c9394755e33cffd861f1c58/pinterest_scraper/scraper.py))

In [0]:
# Pinterest Login

def pinterest_login(userid, password):
  
  browser.get("https://www.pinterest.com")
  email_field = browser.find_element_by_name('id')
  email_field.send_keys(userid) #User ID
  password_field = browser.find_element_by_name('password')
  password_field.send_keys(password) #Passowrd
  password_field.send_keys(Keys.RETURN)
  time.sleep(3)
  
pinterest_login('yashsrivastava3005@gmail.com', 'Yash@1234')

## Helper Functions

### Scrape Pin IDs

1) How to scroll to the end of the page using selenium in python ([StackOverFlow](https://stackoverflow.com/questions/32391303/how-to-scroll-to-the-end-of-the-page-using-selenium-in-python/32629481))

In [0]:
def get_pinterest_ids(user):
  
  ids = []
  browser.get('https://www.pinterest.com/'+user+'/pins/')
  #number_of_pins_tag = browser.find_elements_by_tag_name('strong')
  #number_of_pins = int(number_of_pins_tag[0].text)
  #print(len(number_of_pins_tag),number_of_pins)
  
  page_length = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var page_length=document.body.scrollHeight;return page_length;")
  flag = False
  tries = 10
  
  while flag==False:
    
    previous_length = page_length
    previous_length_ids = len(set(ids))
    
    page_length = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var page_length=document.body.scrollHeight;return page_length;")
    
    #If the previous length is equal to current length, that means we are at the end of the page.
    if previous_length==page_length:
      flag = True
      
    #Sleep Time added so fake that Instagram page is not being traversed by a Bot.
    time.sleep(5)
  
    a_tags = browser.find_elements_by_tag_name('div')
      
    for link in a_tags:
      
      temp = link.get_attribute('data-test-pin-id')
      
      if temp!=None:
        ids.append(temp)
    
    print(len(set(ids)), previous_length_ids)
    
    if len(set(ids))==previous_length_ids:
      
      tries = tries -1
      
      if tries>=0:
        flag = False
      
      else:
        flag = True
        
    else:
      flag = False
  
  return list(set(ids))

### Retreive Needed Information from JSON Data

In [0]:
def extract_needed_info(json_data):
  
  keys = ['domain', 'id', 'link', 'comment_count', 'category', 'is_video', 'created_at', 'aggregated_pin_data', 'images']
  temp_dict = {}
  
  for key in keys:
    
    if key=='aggregated_pin_data':
      
      temp_key = 'saves'
      temp_dict[temp_key] = json_data[key]['aggregated_stats'][temp_key]
      
    elif key=='images':
      
      temp_key = 'url'
      temp_dict[key+temp_key] = json_data[key]['736x'][temp_key]
    
    else:
      temp_dict[key] = json_data[key]
  
  return temp_dict

### Save Data

In [0]:
def save_metadata(ids, save_name):
  
  data = pd.DataFrame()
  counter = 0
  total_ids = len(ids)
  
  while counter!=total_ids:
  
    for each_id in ids:
      
          link = 'https://www.pinterest.com/pin/' + each_id + '/'
          print(len(ids))
    
          try:
          
            # Open the URL and extract JSON information.
            page = urlopen(link).read()
            source = BeautifulSoup(page, 'html.parser')
            body = source.find('body')
            script = body.find('script')
            json_data = json.loads(script.text)

            # Extract needed information.
            image_metadata = json_data['resources']['data']['PinPageResource']["field_set_key=\"detailed\",get_page_metadata=true,id=\""+ each_id +"\",main_module_name=\"UnauthPinReactPage\",pure_react=true,python_resource_prefetch=false"]['data']
            temp_dict = extract_needed_info(image_metadata)
            temp = pd.DataFrame(temp_dict, index=[0])
            data = data.append(temp)
            ids.remove(each_id)
            counter = counter + 1
            
          except Exception as error:
            print(link)
            print('Caught this error: ' + repr(error))
  
  data = data.drop_duplicates(subset = 'id')
  data.index = range(len(data.index))
  data.to_csv(save_name+'data.csv')
  save_images(data, save_name)

### Save Images

References:

1) Scraping Instagram with Python (using Selenium and Beautiful Soup) ([Medium](https://medium.com/@srujana.rao2/scraping-instagram-with-python-using-selenium-and-beautiful-soup-8b72c186a058))

In [0]:
#The Code has been taken from the above resource.

def save_images(data, save_name):
  
  images_path = save_name+'/'
  
  if not os.path.isdir(images_path):
    os.mkdir(images_path)
  
  for i in range(0, len(data['imagesurl'])):
    
    image = requests.get(data['imagesurl'][i])
    with open(images_path + data['id'][i] + ".jpg", 'wb') as file:
                    file.write(image.content)

## Scrape Pinterest Pins

In [0]:
for user in pinterest_users:
  
  ids = get_pinterest_ids(user)
  print(len(ids))
  with open(user + '_ids.pkl', 'wb') as f:
    pickle.dump(ids, f)
  #save_metadata(ids, user+'_pins_')
  #shutil.make_archive(user+'_pins_images', 'zip', user+'_pins_')
  #break

7175 7158
7190 7175
7206 7190
7223 7206
7237 7223
7253 7237
7268 7253
7286 7268
7301 7286
7318 7301
7335 7318
7351 7335
7367 7351
7384 7367
7401 7384
7416 7401
7434 7416
7451 7434
7467 7451
7484 7467
7500 7484
7517 7500
7534 7517
7551 7534
7568 7551
7584 7568
7600 7584
7617 7600
7633 7617
7650 7633
7665 7650
7682 7665
7697 7682
7712 7697
7727 7712
7743 7727
7760 7743
7776 7760
7791 7776
7806 7791
7822 7806
7838 7822
7854 7838
7870 7854
7888 7870
7905 7888
7924 7905
7940 7924
7957 7940
7975 7957
7994 7975
8010 7994
8028 8010
8045 8028
8062 8045
8078 8062
8093 8078
8109 8093
8124 8109
8142 8124
8157 8142
8173 8157
8189 8173
8207 8189
8223 8207
8242 8223
8259 8242
8276 8259
8291 8276
8307 8291
8324 8307
8340 8324
8356 8340
8373 8356
8390 8373
8408 8390
8425 8408
8441 8425
8459 8441
8474 8459
8490 8474
8506 8490
8522 8506
8538 8522
8555 8538
8571 8555
8588 8571
8605 8588
8622 8605
8639 8622
8656 8639
8673 8656
8690 8673
8708 8690


StaleElementReferenceException: ignored